# Swap

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) and the Refinitiv Data Libraries.

##### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
 and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

#### Import the library and connect to the Refinitv Data Libraries


In [1]:
import refinitiv.data as rd
from refinitiv.data.content.ipa.financial_contracts import swap
import pandas as pd

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x1038518e0 {name='workspace'}>

### Custom Schedule for  Amortization

This sample generate the cash flows of an interest rate swap whose principal amortizes according to a custom schedule. The 'fields' requested are:
- "CashFlowDatesArray": the cash flow payment date
- "CashFlowInterestAmountsInDealCcyArray" the interet amount paid or received
- "CashFlowResidualAmountsInLegCcyArray", the remaining pricipal at each payment date

the amortization parameters is based on two amortization sections for each leg that contains:
- "startDate": the start date of the amortization period. "endDate" isn't necessary as a period ends when the next one starts or when the swap terminates
- "amount": this is the amount that is amortized at each amortization date
- "AmortizationFrequency": the frequency of amortization (EveryCoupon, Every2ndCoupon, Every3rdCoupon, Every4thCoupon, Every12thCoupon)
- "AmortizationType": it is set to "Schedule".


In [3]:
response = swap.Definition(
    instrument_tag="IRS-EURIBOR-5Y",
    start_date="2020-01-28",
    tenor="5Y",
    legs=[
        swap.LegDefinition(
            direction=swap.Direction.PAID,
            notional_amount=10000000.0,
            notional_ccy="EUR",
            interest_payment_frequency=swap.Frequency.ANNUAL,
            interest_type=swap.InterestType.FIXED,
            fixed_rate_percent=0.5,
            interest_calculation_method="Dcb_30_360",
            amortization_schedule=[
                swap.AmortizationItem(
                    start_date="2020-01-28",
                    end_date="2022-01-28",
                    amount=500000,
                    amortization_frequency=swap.AmortizationFrequency.EVERY_COUPON,
                    amortization_type=swap.AmortizationType.SCHEDULE),
                swap.AmortizationItem(
                    start_date="2022-01-28",
                    end_date="2025-01-28",
                    amount=3000000,
                    amortization_frequency=swap.AmortizationFrequency.EVERY_COUPON,
                    amortization_type=swap.AmortizationType.SCHEDULE)
            ],
        ),
        swap.LegDefinition(
            direction=swap.Direction.RECEIVED,
            notional_amount=10000000.0,
            notional_ccy="EUR",
            interest_payment_frequency=swap.Frequency.SEMI_ANNUAL,
            index_name="EURIBOR",
            interest_type=swap.InterestType.FLOAT,
            interest_calculation_method="Dcb_Actual_360",
            index_tenor="6M",
            amortization_schedule=[
                swap.AmortizationItem(
                    start_date="2020-01-28",
                    end_date="2022-01-28",
                    amount=250000,
                    amortization_frequency=swap.AmortizationFrequency.EVERY_COUPON,
                    amortization_type=swap.AmortizationType.SCHEDULE),
                swap.AmortizationItem(
                    start_date="2022-01-28",
                    end_date="2025-01-28",
                    amount=1500000,
                    amortization_frequency=swap.AmortizationFrequency.EVERY_COUPON,
                    amortization_type=swap.AmortizationType.SCHEDULE)
            ],
        ),
    ],
    pricing_parameters=swap.PricingParameters(
        valuation_date="2020-10-20T00:00:00Z"
    ),
    fields=["InstrumentTag",
            "Direction",
            "CashFlowDatesArray",
            "CashFlowInterestAmountsInDealCcyArray",
            "CashFlowResidualAmountsInLegCcyArray",
            "DiscountCurveName",
            "ForwardCurveName",
            "ErrorCode",
            "ErrorMessage"],
).get_data().data.df

response

,InstrumentTag,Direction,CashFlowDatesArray,CashFlowInterestAmountsInDealCcyArray,CashFlowResidualAmountsInLegCcyArray,DiscountCurveName,ForwardCurveName,ErrorCode,ErrorMessage
0,IRS-EURIBOR-5Y,Paid,"[2021-01-28T00:00:00Z, 2022-01-28T00:00:00Z, 2...","[-50000.0, -47500.0, -45250.0, -29916.66666666...","[9500000.0, 9000000.0, 6000000.0, 3000000.0, 0.0]",EUR - Swap vs 6M Euribor,<NA>,,
1,IRS-EURIBOR-5Y,Received,"[2020-07-28T00:00:00Z, 2021-01-28T00:00:00Z, 2...","[39147.743978012455, 36096.82382594561, 28663....","[9750000.0, 9500000.0, 9250000.0, 9000000.0, 7...",EUR - Swap vs 6M Euribor,EUR - Swap vs 6M Euribor,,


In [4]:
paid_cash_flow_df = pd.DataFrame()

paid_cash_flow_df['Paid Dates'] = response[response["Direction"]
                                           == "Paid"]['CashFlowDatesArray'][0]
paid_cash_flow_df['Paid InterestPercent'] = response[response["Direction"]
                                                     == "Paid"]['CashFlowInterestAmountsInDealCcyArray'][0]
paid_cash_flow_df['Remaining Principal'] = response[response["Direction"]
                                                    == "Paid"]['CashFlowResidualAmountsInLegCcyArray'][0]

paid_cash_flow_df

,Paid Dates,Paid InterestPercent,Remaining Principal
0,2021-01-28T00:00:00Z,-50000.000000,9500000.0
1,2022-01-28T00:00:00Z,-47500.000000,9000000.0
2,2023-01-30T00:00:00Z,-45250.000000,6000000.0
3,2024-01-29T00:00:00Z,-29916.666667,3000000.0
4,2025-01-28T00:00:00Z,-14958.333333,0.0


In [5]:
received_cash_flow_df = pd.DataFrame()

received_cash_flow_df['Rec Dates'] = response[response["Direction"]
                                              == "Received"]['CashFlowDatesArray'][1]
received_cash_flow_df['Rec InterestPercent'] = response[response["Direction"]
                                                        == "Received"]['CashFlowInterestAmountsInDealCcyArray'][1]
received_cash_flow_df['Remaining Principal'] = response[response["Direction"]
                                                        == "Received"]['CashFlowResidualAmountsInLegCcyArray'][1]

received_cash_flow_df

,Rec Dates,Rec InterestPercent,Remaining Principal
0,2020-07-28T00:00:00Z,39147.743978,9750000.0
1,2021-01-28T00:00:00Z,36096.823826,9500000.0
2,2021-07-28T00:00:00Z,28663.562869,9250000.0
3,2022-01-28T00:00:00Z,27082.707641,9000000.0
4,2022-07-28T00:00:00Z,25633.970341,7500000.0
5,2023-01-30T00:00:00Z,22338.781478,6000000.0
6,2023-07-28T00:00:00Z,17840.162767,4500000.0
7,2024-01-29T00:00:00Z,14512.015101,3000000.0
8,2024-07-29T00:00:00Z,10102.138764,1500000.0
9,2025-01-28T00:00:00Z,5444.146543,0.0


In [6]:
rd.close_session()